# Gradio Web Application for Adapter-Tuned (LoRA) Model
## Objective
The objective of this notebook is to test and launch a live, interactive web application for the Adapter-Tuned (LoRA) NLLB translation model. It contains the complete, self-contained Python code required to build a Gradio-based user interface that allows users to perform bidirectional Odia-German translation.

## Methodology
The script is designed to be a complete web application that showcases the unique loading process for a PEFT model.

1. **Model Loading:** It first loads the original, large **base NLLB model** in 8-bit precision. It then loads the small, fine-tuned **LoRA adapters** from their repository on the Hugging Face Hub and applies them to the base model.
2. **Language Detection:** It implements a robust, hybrid language detection system, prioritizing a script-based check for Odia and using the `langdetect` library as a fallback.
3. **Translation Logic:** It defines a central `translate_text` function that takes user input, runs the detection logic, and calls the loaded model to generate the translation.
4. **Web Interface:** It uses the `gradio` library to create a clean user interface, complete with text boxes, a dropdown for manual language selection, and example sentences.

## Workflow
1. Installs all required libraries (`gradio`, `transformers`, `peft`, etc.).
2. Loads the base NLLB model and the LoRA adapters from the Hub.
3. Defines the language detection and translation functions.
4. Creates the Gradio `Interface` object.
5. Launches the web application, creating a temporary public URL for testing in the Colab environment.

## Input & Output
* **Input:** Text entered by a user into the Gradio web interface.
* **Output:** A live, interactive Gradio web application for bidirectional Odia-German translation powered by the LoRA fine-tuned model.

In [ ]:
# Uninstall all potentially conflicting packages
!pip uninstall -y torch torchvision torchaudio transformers gradio langdetect sentencepiece accelerate huggingface-hub safetensors peft bitsandbytes torchtune sentence-transformers timm

In [ ]:
# Clear pip cache
!pip cache purge

In [ ]:
# Install libraries
!pip install torch==2.6.0+cu124 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers==4.52.4
!pip install gradio==5.31.0
!pip install langdetect==1.0.9
!pip install sentencepiece==0.2.0
!pip install huggingface-hub==0.33.0
!pip install accelerate==1.7.0
!pip install safetensors==0.5.3
!pip install peft==0.15.2
!pip install bitsandbytes==0.46.1

In [ ]:
# Verify installations
!pip show torch transformers gradio langdetect sentencepiece huggingface-hub accelerate safetensors bitsandbytes

In [ ]:
# Check CUDA availability and version
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Version: {torch.version.cuda}")

## Restart the Runtime. Then execute the code below.

In [ ]:
# clear cache
!rm -rf ~/.cache/huggingface

In [ ]:
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Version: {torch.version.cuda}")

In [ ]:
# import libraries
import gradio as gr
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from langdetect import detect, LangDetectException
import torch
import traceback
import logging
import re

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# load Adapter-based fine-tuned model (from the hub)
BASE_MODEL_HUB_ID = "facebook/nllb-200-distilled-600M"
ADAPTER_MODEL_HUB_ID = "abhinandansamal/nllb-200-distilled-600M-LoRA-finetuned-odia-german-bidirectional"
ODIA_LANG_CODE = "ory_Orya"
GERMAN_LANG_CODE = "deu_Latn"

In [ ]:
print("Loading the Adapter-based fine-tuned bidirectional model...")
translator = None
try:
  # Step A: Load the original base model (quantized for efficiency)
  print(f"Loading base model: {BASE_MODEL_HUB_ID}...")
  bnb_config = BitsAndBytesConfig(load_in_8bit=True)
  base_model = AutoModelForSeq2SeqLM.from_pretrained(
      BASE_MODEL_HUB_ID,
      quantization_config=bnb_config,
      device_map="auto"
  )

  # Load the NLLB tokenizer
  tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_HUB_ID, src_lang=ODIA_LANG_CODE, tgt_lang=GERMAN_LANG_CODE)

  # Step B: Load the LoRA adapters and apply them to the base model
  print(f"Loading and applying adapters from: {ADAPTER_MODEL_HUB_ID}...")
  model = PeftModel.from_pretrained(base_model, ADAPTER_MODEL_HUB_ID)
  model.eval() # Set the model to evaluation mode

  # Step C: Create the pipeline with the complete, merged model
  print("Creating translation pipeline...")
  translator = pipeline(
      "translation",
      model=model,
      tokenizer=tokenizer,
      src_lang=ODIA_LANG_CODE, # Provide defaults for the pipeline
      tgt_lang=GERMAN_LANG_CODE
  )
  print("✅ Model and pipeline loaded successfully!")

except Exception as e:
  print(f"❌ Detailed error: {traceback.format_exc()}")
  translator = None

In [ ]:
# Script-based Odia detection
def is_odia_text(text):
  """
  Checks if the input text contains Odia script characters.

  This function uses a regular expression to detect the presence of characters in the Unicode
  range for Odia script (U+0B00–U+0B7F). It logs the detection result and returns True if Odia
  characters are found, False otherwise. Empty or whitespace-only inputs are considered invalid.

  Args:
    text (str): The input text to check for Odia script characters.

  Returns:
    bool: True if the text contains at least one Odia script character, False otherwise.

  Example:
    >>> import logging
    >>> logging.basicConfig(level=logging.INFO)
    >>> logger = logging.getLogger()
    >>> is_odia_text("ନମସ୍କାର")
    INFO:root:Odia script detection for 'ନମସ୍କାର...': True
    True
    >>> is_odia_text("Hallo")
    INFO:root:Odia script detection for 'Hallo...': False
    False
    >>> is_odia_text("")
    False
  """
  if not text or not text.strip():
    return False

  odia_pattern = re.compile(r'[\u0B00-\u0B7F]')

  match = odia_pattern.search(text)

  logger.info(f"Odia script detection for '{text[:50]}...': {bool(match)}")

  return bool(match)

In [ ]:
# Translation logic
def translate_text(input_text, source_lang="auto"):
  """
  Translates text between Odia and German, with automatic or manual language detection.

  This function translates input text using a preloaded translation model, supporting Odia-to-German
  and German-to-Odia directions. If `source_lang` is 'auto', it prioritizes script-based detection
  for Odia using `is_odia_text`, falling back to `langdetect` for non-Odia text. It logs the process
  and handles errors gracefully, returning error messages for invalid inputs or failures.

  Args:
    input_text (str): The text to translate.
    source_lang (str, optional): Source language code ('auto', 'or' for Odia, 'de' for German).
                                 Defaults to 'auto' for automatic detection.

  Returns:
    str: The translated text or an error message if translation or language detection fails.
  """
  if translator is None:
    return "Error: Model could not be loaded."
  if not input_text or not input_text.strip():
    return "Error: Input text is empty."

  try:
    if source_lang != "auto":
      detected_lang = source_lang
      logger.info(f"Manual source language selected: {detected_lang}")
    else:
      # Prioritize script detection for Odia
      if is_odia_text(input_text):
        detected_lang = "or"
        logger.info(f"Detected language (script-based): {detected_lang}")
      else:
        # Use langdetect for non-Odia text
        try:
          detected_lang = detect(input_text)
          logger.info(f"Detected language (langdetect): {detected_lang}")
        except LangDetectException as e:
          logger.error(f"LangDetectException: {e}")
          return "Error: Could not detect language. Please select Odia or German manually."
        except Exception as e:
          logger.error(f"Unexpected error in langdetect: {e}")
          return "Error: Language detection failed. Please select Odia or German manually."

    if detected_lang == "or":
      result = translator(input_text, src_lang=ODIA_LANG_CODE, tgt_lang=GERMAN_LANG_CODE, max_length=1024)
      logger.info(f"Translating Odia to German: {input_text[:50]}...")
    elif detected_lang == "de":
      result = translator(input_text, src_lang=GERMAN_LANG_CODE, tgt_lang=ODIA_LANG_CODE, max_length=1024)
      logger.info(f"Translating German to Odia: {input_text[:50]}...")
    else:
      logger.warning(f"Unsupported language detected: {detected_lang}")
      return f"Error: Translation from '{detected_lang}' is not supported. Please select Odia or German."

    return result[0]["translation_text"]
  except Exception as e:
    logger.error(f"Translation error: {traceback.format_exc()}")
    return f"Error: Translation failed - {str(e)}"

In [ ]:
# Test translation
if translator is None:
  print("Translator not initialized due to previous errors.")
else:
  odia_text = "କମ୍ପ୍ୟୁଟର ଆଧାରିତ ଏହି ପରୀକ୍ଷାର ଫଳାଫଳ ୧୫ ଜୁଲାଇରେ ଘୋଷଣା ହେବାର ଆଶା କରାଯାଉଛି।"
  try:
    translation = translate_text(odia_text, source_lang="auto")
    print(f"Odia: {odia_text}")
    print(f"German: {translation}")
  except Exception as e:
    print(f"❌ Error during translation: {e}")

In [ ]:
# Create Gradio interface
title = "LoRA Bidirectional Odia ↔ German Translator"
description = """
An Adapter-Based (LoRA) Fine-tuned NLLB model for translating between Odia and German.
Enter text and select the source language (or use auto-detection).
"""
examples = [
    ["ଆଜି ପାଗ ବହୁତ ଭଲ ଅଛି।", "or"],  # "The weather is very nice today."
    ["Die derzeitige Wachstumsrate von 6,5 Prozent ist sehr lobenswert.", "de"]  # "The current growth rate of 6.5 percent is very commendable."
]

iface = gr.Interface(
    fn=translate_text,
    inputs=[
        gr.Textbox(lines=5, label="Input Text (Odia or German)", placeholder="Type or paste text here..."),
        gr.Dropdown(choices=["auto", "or", "de"], label="Source Language", value="auto")
    ],
    outputs=gr.Textbox(lines=5, label="Translation"),
    title=title,
    description=description,
    examples=examples,
    allow_flagging="never",
    theme=gr.themes.Soft()
)

In [ ]:
# Launch in Colab
print("\nLaunching Gradio interface...")
iface.launch(share=True)